In [1]:
import sys
import os

import geopandas as gpd
import pandas as pd

from osgeo import gdal
from pyproj import CRS

In [25]:
# load raw vector data
db_data = gpd.read_file("/home/philipp/Data/edin_diss/GIS_tax/raw_vector_Stichtag_20200228.gdb", layer="Waldorte")

In [26]:
# define crs of orthophotos
new_crs = CRS.from_user_input('PROJCS["Austria_Lambert",GEOGCS["GCS_BESSEL_AUT",DATUM["D_BESSEL_AUT",SPHEROID["Bessel_1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.333333333],PARAMETER["standard_parallel_1",46],PARAMETER["standard_parallel_2",49],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')
# reproject to new crs
db_data = db_data.to_crs(new_crs)

In [27]:
# add year the data is valid from
db_data['year_data'] = 2019

In [5]:
# save to file
db_data.to_file("/home/philipp/Data/edin_diss/GIS_tax/gis_wo/gis_wo.shp")

In [28]:
db_data.head()

,FB_KURZ,FB_LANG,FR_KURZ,FR_LANG,ABTEILUNG,UNTERABTEILUNG,FARBCODE,LINKID,SHAPE_Length,SHAPE_Area,geometry,year_data
0,179,Pongau,8,Gastein,647,C2,12000000000,{770B160D-349F-4E03-8E31-C09966044E95},1490.713604,31382.525999,"MULTIPOLYGON (((386279.300 358369.925, 386291....",2019
1,179,Pongau,1,Filzmoos,376,G4,12000000000,{B0F0509B-37B0-42AD-866E-4B6D75DF4AA6},377.331016,7873.920587,"MULTIPOLYGON (((411127.657 394260.340, 411148....",2019
2,179,Pongau,8,Gastein,689,I0,15000000000,{05C33B9A-5FEC-4754-97D8-D01CF5BDDD1C},466.808748,10235.032218,"MULTIPOLYGON (((380930.470 358686.725, 380935....",2019
3,179,Pongau,4,Kleinarl,554,G0,35200300000,{6E9C11FE-55F5-4247-A2B5-202F122286F6},1432.461887,39275.813450,"MULTIPOLYGON (((397372.350 374393.005, 397368....",2019
4,179,Pongau,3,Flachau,224,v_8/001,00000000008,{75650917-8ACF-46EF-912F-201029A152B2},2062.772651,27072.555584,"MULTIPOLYGON (((402954.780 381514.065, 402947....",2019


In [8]:
db_data.crs

<Projected CRS: PROJCS["Austria_Lambert",GEOGCS["GCS_BESSEL_AUT",D ...>
Name: Austria_Lambert
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: D_BESSEL_AUT
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [29]:
db_data.columns

Index(['FB_KURZ', 'FB_LANG', 'FR_KURZ', 'FR_LANG', 'ABTEILUNG',
       'UNTERABTEILUNG', 'FARBCODE', 'LINKID', 'SHAPE_Length', 'SHAPE_Area',
       'geometry', 'year_data'],
      dtype='object')

In [32]:
db_data['id'] = db_data['FB_KURZ'].astype(str) + db_data['FR_KURZ'].astype(str) + \
                db_data['ABTEILUNG'].astype(str) + db_data['UNTERABTEILUNG'].astype(str)

In [33]:
db_data['id']

0              1798647C2
1              1791376G4
2              1798689I0
3              1794554G0
4         1793224v_8/001
               ...      
373121         8411480B2
373122    8411209v_8/006
373123         8411172A2
373124         8411511P5
373125     841150v_3/001
Name: id, Length: 373126, dtype: object

In [2]:
# load data about SAP tax
sap_path = "/home/philipp/Data/edin_diss/SAP_tax"
sap_tax = pd.read_csv(sap_path + "/sap_meta.csv", sep=',')

year_data = '2019'

In [3]:
sap_tax.head()

,FB,FR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,171,1,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
1,171,2,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
2,171,3,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
3,171,4,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
4,171,5,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356


In [4]:
# get all 
tos = sap_tax[year_data].unique()
tos

array([1030, 1353, 1049, 1053, 1052, 1100, 1050, 1051, 1083, 1042, 1073,
       1092, 1311, 1342, 1151, 1194, 1086, 1043, 1144, 1197, 1055, 1303,
       1085, 1345, 1302, 1140, 1312, 1065, 1066, 1067, 1088, 1044, 1196,
       1313, 1343, 1354, 1093, 1045, 1081, 1082, 1341, 1355, 1156, 1250])

In [5]:
# create df for giveb year
sap_tax_year = sap_tax[['FB', year_data]]

In [6]:
def concat_sap_tax(tos, sap_tax_year):
    
    sap_tax_tos = []

    for to in tos:
        # find Forstbetrieb for the giveb Teiloperat
        fb = sap_tax_year.loc[sap_tax_year[year_data] == to, 'FB'].unique()[0]
        path = sap_path + "/" + str(fb) + "/TO_" + str(to) + ".XLS"
        sap_tax_tos.append(pd.read_csv(path, sep='\t', encoding = "ISO-8859-1", decimal=',', error_bad_lines=False, na_values=0))
        print(path)

    return pd.concat(sap_tax_tos)

In [8]:
sap_tax = concat_sap_tax(tos, sap_tax_year)

/home/philipp/miniconda3/envs/geopd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (2,4,29,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS


/home/philipp/miniconda3/envs/geopd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/171/TO_1353.XLS


/home/philipp/miniconda3/envs/geopd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (0,2,4,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


...

/home/philipp/Data/edin_diss/SAP_tax/179/TO_1313.XLS
/home/philipp/Data/edin_diss/SAP_tax/180/TO_1343.XLS


/home/philipp/miniconda3/envs/geopd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (40,146,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/180/TO_1354.XLS


/home/philipp/miniconda3/envs/geopd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (29,146,147,148) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/181/TO_1093.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1045.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1081.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1082.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1341.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1355.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1156.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1250.XLS


In [9]:
sap_tax.shape

(1098308, 152)

In [10]:
# filter for Durchforstungen

In [11]:
sap_tax.columns[:100]

Index(['Merkmalausprägung', 'AuswKatTyp', 'Teiloperats-ID', 'Forstbetrieb',
       'Debitor', 'TO-Bezeichnung', 'Status', 'Beg. Laufzeit', 'Ende Laufzeit',
       'Operat-ID', 'vorgeschl. Hiebssatz', 'Verantwortlicher',
       'Erfassungsstatus', 'Migriert?', 'GUID', 'Forstbetrieb.1',
       'Teiloperats-ID.1', 'Forstrevier', 'Abteilung', 'Unterabteil.',
       'Teilfl.', 'Debitor.1', 'Bearbeitungsblock', 'WE-Typ', 'Betriebsklasse',
       'Umtriebszeit', 'Nebengrund Art', 'Ertragssituation',
       'Bewirtschaftungsform', 'Schutzwaldkategorie', 'Fläche in HA',
       'Seehöhe', 'Exposition', 'Neigung', 'Standorteinheit', 'Vegetationstyp',
       'Waldtyp', 'Wuchsgebiet', 'Überh. Laubholz', 'Überh. Nadelhz.',
       'fr. Schälschade', 'Verbissgrad', 'SchutzwaldProjNr', 'Schlussgrad',
       'Stabilität', 'VJ Bedingung', 'VJ Situation', 'Erreichbark. des BZ',
       'Selektiver Verbiss', 'Erfassungsstatus.1', 'Storno', 'Angelegt von',
       'Angelegt am', 'Uhrzeit', 'Geändert von', 'Ge

In [12]:
sap_tax.columns[100:]

Index(['Erfassungsstatus.3', 'Storno.2', 'GUID.5', 'GUID.6', 'Forstbetrieb.4',
       'Teiloperats-ID.4', 'Forstrevier.3', 'Abteilung.3', 'Unterabteil..3',
       'Teilfl..3', 'Best.-Schicht.2', 'Nutzungsnummer', 'Maßnahmenart',
       'Massnahme geplant', 'Massnahmengruppe', 'Angriffsfläche', 'Nutzung LH',
       'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit', 'Bewpfl.',
       'Zeitpunkt', 'Rückungsart', 'Schlägerungsart', 'Erfassungsstatus.4',
       'Storno.3', 'Nutztext', 'Alter der 1. Schicht', 'TAX: Altersklasse',
       'Repr. Fläche Schicht', 'Produktionskategorie', 'Geschäftsjahr',
       'Abmaßbeleg', 'Maßnahme', 'Geschäftsfeld', 'Bezeichnung',
       'Pflanzen Ist', 'Baumarten Ist', 'Repr. Fläche Baumart',
       'Ertragstafelnummer', 'Ertragstafelbezeich', 'Anmerkung', 'Zeile1',
       'Zeile2', 'Zeile3', 'Zeile4', 'Zeile5', 'Zeile6', 'Zeile7', 'Zeile8',
       'Bestockungsziel', 'Flächenanteil'],
      dtype='object')

In [13]:
sap_tax.fillna(0, inplace=True)

In [39]:
sap_tax['Forstbetrieb'] = sap_tax['Forstbetrieb'].astype(int)
sap_tax['Forstrevier'] = sap_tax['Forstrevier'].astype(int)
sap_tax['Abteilung'] = sap_tax['Abteilung'].astype(int)
sap_tax['Teilfl.'] = sap_tax['Teilfl.'].astype(int)
sap_tax['Nutzdringlichkeit'] = sap_tax['Nutzdringlichkeit'].astype(int)
sap_tax['year_to_start'] = sap_tax['Beg. Laufzeit'].str[-4:].astype(int)
sap_tax['year_to_end'] = sap_tax['Ende Laufzeit'].str[-4:].astype(int)

ValueError: cannot convert float NaN to integer

In [20]:
# create ID_wo

sap_tax['id'] = sap_tax['Forstbetrieb'].astype(str) + sap_tax['Forstrevier'].astype(str) + \
sap_tax['Abteilung'].astype(str) + sap_tax['Unterabteil.'].astype(str) + sap_tax['Teilfl.'].astype(str)

In [21]:
sap_tax_df = sap_tax.loc[ (sap_tax['Maßnahmenart'] == 'DE') | (sap_tax['Maßnahmenart'] == 'DF') \
                         | (sap_tax['Maßnahmenart'] == 'ND'), ['id', 'Forstbetrieb', 'Forstrevier', \
                        'Abteilung', 'Unterabteil.', 'Teilfl.', 'Maßnahmenart', 'Nutzdringlichkeit']]

In [22]:
sap_tax_df

,id,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Maßnahmenart,Nutzdringlichkeit
10,1711652C1,171,1,652,C,1,DF,2
26,1711658A1,171,1,658,A,1,DF,2
38,1711599G1,171,1,599,G,1,DF,2
95,1711726D1,171,1,726,D,1,DF,2
149,1711520E1,171,1,520,E,1,DF,2
...,...,...,...,...,...,...,...,...
16020,182863F2,182,8,63,F,2,DE,2
16076,1829365B2,182,9,365,B,2,DE,2
16174,1829350H2,182,9,350,H,2,DE,2
16192,182822A3,182,8,22,A,3,DE,1


In [34]:
sap_tax[['Beg. Laufzeit', 'Ende Laufzeit']]

,Beg. Laufzeit,Ende Laufzeit
0,01.01.2009,31.12.2019
1,01.01.2009,31.12.2019
2,01.01.2009,31.12.2019
3,01.01.2009,31.12.2019
4,01.01.2009,31.12.2019
...,...,...
16210,01.01.2016,31.12.2025
16211,01.01.2016,31.12.2025
16212,01.01.2016,31.12.2025
16213,01.01.2016,31.12.2025


In [40]:
sap_tax['Beg. Laufzeit'].str[-4:].astype(int)

ValueError: cannot convert float NaN to integer

In [44]:
sap_tax['Beg. Laufzeit'].unique()

array(['01.01.2009', 0, '01.01.2019', '01.01.2010', '01.01.2011',
       '01.01.2012', '01.01.2013', '01.01.2014', '01.01.2017',
       '01.01.2018', '01.01.2015', '01.01.2016', '01.01.2007',
       '01.01.2008'], dtype=object)

In [45]:
sap_tax[sap_tax['Beg. Laufzeit'] == 0]

,Merkmalausprägung,AuswKatTyp,Teiloperats-ID,Forstbetrieb,Debitor,TO-Bezeichnung,Status,Beg. Laufzeit,Ende Laufzeit,Operat-ID,...,Zeile2,Zeile3,Zeile4,Zeile5,Zeile6,Zeile7,Zeile8,Bestockungsziel,Flächenanteil,id
4405,0,120.0,VI,0,HB,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
19532,N-Tl möglichst dich halten - keine unnötigen L...,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
19533,0,125.0,VII,0,HB,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
171,BZ: 1 BU 7 FI 2 LÄ,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
172,BZ: 1 BU 7 FI 2 LÄ,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
173,BZ: 1 BU 7 FI 2 LÄ,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
174,BZ: 1 BU 7 FI 2 LÄ,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
175,0,140.0,VII,0,HB,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
177,BZ: 1 BU 7 FI 2 LÄ,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
178,BZ: 1 BU 7 FI 2 LÄ,0.0,0,0,0,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,00000
